<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [2]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [3]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [4]:
# method most_simmilar
wv_embeddings.most_similar('dog', topn=5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

***Ваш ответ:*** 'cat' не входит в топ-5 ближайших слов к 'dog'. Входит слово **cats**

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [4]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\mserg\AppData\Local\Temp\ipykernel_22548\2760772041.py:9: SyntaxWarning: invalid escape sequence '\w'
  return re.findall('\w+', text)


In [5]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    mean = np.zeros(shape=dim)
    size = 0
    words = tokenizer.tokenize(question.lower())
    for word in words:
      if word in embeddings:
         mean += embeddings[word]
         size += 1
    return mean / size if size != 0 else mean


Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [7]:
# Предложение
question = "I love neural networks"

round(question_to_vec(question, wv_embeddings, tokenizer)[2], 2)

-1.29

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0 \\
   0, &x \geq 0
 \end{cases}
\end{equation}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)



[K=10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}}\approx  0.3$

#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?



$$ \text{Hits@47} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le 47],$$
$$ \text{DCG@1} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le 1].$$
Максимум будет достигнут при двух условиях:
1. $ \text{rank}_{q'_i} \leq 47 \,\,\,\forall  i \in [1, N], N <= K$ для функции Hits@47
2. $ \text{rank}_{q'_i} > 1 \,\,\,\forall  i \in [1, 1]$ для функции DCG@1

Т.е при выполнении первого условия по определению индикаторной функции сумма будет равна N, что в итоге даёт $\large \frac{N}{N}=1$, и при выполнении второго условия по определению индикаторной функции сумма будет равняться 0, что в итоге даёт $\large 0\times\frac{1}{N}$ => Максимум будет равен $1-0=1$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [6]:


def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Подсчитываем количество дубликатов, чей ранг <= k

    return sum([0 if rank > k else 1 for rank in dup_ranks]) / len(dup_ranks)

In [9]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [7]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    # Вычисляем сумму для всех релевантных дубликатов
    summa = sum([0 if rank > k else math.log2(1+rank)**(-1) for rank in dup_ranks])

    # Делим на общее количество вопросов
    dcg_value = summa / len(dup_ranks)
    return dcg_value

In [11]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [8]:
import pandas as pd

In [13]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [14]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

Считайте данные.

In [9]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [10]:
validation_data = read_corpus('datasets/validation.tsv')

Кол-во строк

In [17]:
len(validation_data)

3760

Размер нескольких первых строк

In [18]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [11]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vec = question_to_vec(question, embeddings, tokenizer, dim)

    candidates_cosine_sim = []
    for candidate in candidates:
        candidates_cosine_sim.append(cosine_similarity(question_vec.reshape(1, -1), question_to_vec(candidate, embeddings, tokenizer, dim).reshape(1, -1))[0][0])

    indexed_candidates_cosine_sim = list(enumerate(candidates_cosine_sim))
    
    sorted_candidates_cosine_sim = sorted(indexed_candidates_cosine_sim, key=lambda x: x[1], reverse=True)

    result = [(index, candidates[index]) for index, _ in sorted_candidates_cosine_sim]

    return deepcopy(result)

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [21]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [22]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()


[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?


021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [25]:
from tqdm.notebook import tqdm

In [24]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [25]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [14]:
train_data = read_corpus('datasets/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [27]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

#### Попробуем подобрать оптимальные значения min_count и window

Также будем использовать стоп слова

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [75]:
def preprocess(text):
    result = ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords]) # из семинара
    return result

In [76]:
# Создаем общий корпус текстов
corpus = []

for couple in train_data:
    combined_text = couple[0] + " " + couple[1]
    corpus.append(preprocess(combined_text).split())

In [77]:
train_data[0]

['converting string to list',
 'Convert Google results object (pure js) to Python object']

In [78]:
print(corpus[0])

['converting', 'string', 'list', 'convert', 'google', 'results', 'object', '(', 'pure', 'js', ')', 'python', 'object']


вроде гуд

In [51]:
from gensim.models import Word2Vec

dcg_score_history = []
hits_count_history = []

for min_count in range(1, 10):
    for window in range(1, 10):
        embeddings_trained = Word2Vec(
            sentences=corpus,        # Корпус токенизированных текстов
            vector_size=200,         # Размерность векторов
            window=window,                # Размер окна контекста
            min_count=min_count,             # Минимальная частота слов
            workers=8                # Количество потоков
        ).wv

        wv_ranking = []
        max_validation_examples = 1000
        tokenizer = MyTokenizer()
        for i, line in enumerate(tqdm(validation_data)):
            if i == max_validation_examples:
                break
            q, *ex = line
            ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
            wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        print(f"min_count = {min_count}, window = {window}")
        for k in tqdm([1, 5, 10, 100, 500, 1000]):
            print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
            dcg_score_history.append(dcg_score(wv_ranking, k))
            hits_count_history.append(hits_count(wv_ranking, k))
        print()
        

  0%|          | 0/3760 [00:00<?, ?it/s]

min_count = 1, window = 1


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.358 | Hits@   1: 0.358
DCG@   5: 0.449 | Hits@   5: 0.534
DCG@  10: 0.474 | Hits@  10: 0.610
DCG@ 100: 0.517 | Hits@ 100: 0.821
DCG@ 500: 0.534 | Hits@ 500: 0.951
DCG@1000: 0.539 | Hits@1000: 1.000



  0%|          | 0/3760 [00:00<?, ?it/s]

...

DCG@   1: 0.417 | Hits@   1: 0.417
DCG@   5: 0.505 | Hits@   5: 0.583
DCG@  10: 0.530 | Hits@  10: 0.659
DCG@ 100: 0.577 | Hits@ 100: 0.884
DCG@ 500: 0.589 | Hits@ 500: 0.978
DCG@1000: 0.591 | Hits@1000: 1.000



  0%|          | 0/3760 [00:00<?, ?it/s]

min_count = 9, window = 9


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.414 | Hits@   1: 0.414
DCG@   5: 0.509 | Hits@   5: 0.591
DCG@  10: 0.533 | Hits@  10: 0.665
DCG@ 100: 0.579 | Hits@ 100: 0.886
DCG@ 500: 0.591 | Hits@ 500: 0.979
DCG@1000: 0.594 | Hits@1000: 1.000



In [ ]:
columns = ['DCG@1','DCG@5','DCG@10','DCG@100','DCG@500','DCG@1000',
           'Hits@1','Hits@5','Hits@10','Hits@100','Hits@500','Hits@1000']
row_list = []
for i in range(1,10):
    for j in range(1,10):
        row_list.append('Min_' + str(i) + '/Win_' + str(j))

df = pd.DataFrame()
m = n = 0
for row in row_list:
    for dcg in columns[:6]:
        df.loc[row, dcg] = dcg_score_history[m]
        m += 1
    for hits in columns[6:]:
        df.loc[row, hits] = hits_count_history[n]
        n += 1

df['SUM'] = df.sum(axis=1)
df

,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
Min_1/Win_1,0.358,0.449294,0.473716,0.516790,0.533568,0.538747,0.358,0.534,0.610,0.821,0.951,1.0,7.144115
Min_1/Win_2,0.375,0.463723,0.488898,0.536146,0.551260,0.554427,0.375,0.540,0.618,0.852,0.970,1.0,7.324453
Min_1/Win_3,0.379,0.471943,0.494429,0.543389,0.557576,0.560668,0.379,0.554,0.624,0.862,0.971,1.0,7.397006
Min_1/Win_4,0.387,0.475923,0.501553,0.549254,0.563779,0.566212,0.387,0.554,0.634,0.866,0.977,1.0,7.461722
Min_1/Win_5,0.394,0.483231,0.509244,0.556287,0.569804,0.572355,0.394,0.564,0.644,0.873,0.976,1.0,7.535921
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Min_9/Win_5,0.396,0.495562,0.519405,0.564440,0.578094,0.580445,0.396,0.580,0.655,0.874,0.978,1.0,7.616945
Min_9/Win_6,0.400,0.497833,0.520297,0.567748,0.580638,0.582883,0.400,0.582,0.652,0.880,0.979,1.0,7.642399
Min_9/Win_7,0.410,0.502007,0.526633,0.573088,0.585400,0.587969,0.410,0.581,0.658,0.882,0.976,1.0,7.692098
Min_9/Win_8,0.417,0.505241,0.529732,0.576538,0.588856,0.591215,0.417,0.583,0.659,0.884,0.978,1.0,7.729582


In [56]:
df[df['SUM'] == df['SUM'].max()]

,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
Min_7/Win_9,0.417,0.509045,0.533038,0.578429,0.591035,0.593165,0.417,0.59,0.664,0.883,0.98,1.0,7.755713


Получается оптимальные значения min_count и window 7 и 9 соответственно

#### Попробуем другой токенизатор

In [ ]:
from nltk.tokenize import WordPunctTokenizer

In [58]:
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=9,                # Размер окна контекста
    min_count=7,             # Минимальная частота слов
    workers=8                # Количество потоков
).wv

wv_ranking = []
max_validation_examples = 1000
tokenizer = WordPunctTokenizer()
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.411 | Hits@   1: 0.411
DCG@   5: 0.491 | Hits@   5: 0.562
DCG@  10: 0.517 | Hits@  10: 0.642
DCG@ 100: 0.562 | Hits@ 100: 0.863
DCG@ 500: 0.576 | Hits@ 500: 0.972
DCG@1000: 0.579 | Hits@1000: 1.000


Результат оказался хуже

#### Нормализация (лемматизация)

In [16]:
import spacy

Для нормализации придётся использовать токенизатор от Spacy

In [17]:
nlp = spacy.load('en_core_web_sm')

In [18]:
def preprocess(text):
    spacy_text = nlp(text)
    print(spacy_text)
    lemma_text = ' '.join([token.lemma_ for token in spacy_text])
    print(lemma_text)
    result = ' '.join([word for word in word_tokenize(lemma_text.lower()) if word not in stopWords]) # из семинара
    print(result)
    return result

In [ ]:
# Создаем общий корпус текстов
corpus = []

for idx, couple in enumerate(train_data):
    combined_text = couple[0] + " " + couple[1]
    corpus.append(preprocess(combined_text).split())
    print(f"Обрабатывается элемент {idx}")


In [20]:
train_data[0]

['converting string to list',
 'Convert Google results object (pure js) to Python object']

In [21]:
corpus[0]

['convert',
 'string',
 'list',
 'convert',
 'google',
 'result',
 'object',
 '(',
 'pure',
 'js',
 ')',
 'python',
 'object']

In [26]:
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=9,                # Размер окна контекста
    min_count=7,             # Минимальная частота слов
    workers=8                # Количество потоков
).wv

wv_ranking = []
max_validation_examples = 1000
tokenizer = MyTokenizer()
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.406 | Hits@   1: 0.406
DCG@   5: 0.498 | Hits@   5: 0.579
DCG@  10: 0.523 | Hits@  10: 0.655
DCG@ 100: 0.568 | Hits@ 100: 0.872
DCG@ 500: 0.582 | Hits@ 500: 0.980
DCG@1000: 0.584 | Hits@1000: 1.000


Результат оказался хуже

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


**Какой принцип токенизации даёт качество лучше и почему?**

Лучшее качество даёт токенизация с помощью регулярных выражений и без лемматизации, т.к таким образом для данного текста получается лучшее ранжирование

**Помогает ли нормализация слов?**

С данным текстом нормализация слов не помогает

**Какие эмбеддинги лучше справляются с задачей и почему?**

Лучше всего справляются эмбеддинги с гиперпараметром min_count = 7, что позволяло пропускать повторяющиеся слова, с гиперпараметром window = 9, что позволяло использовать большое сколзящее окно, и с отсутствием нормализации.

**Почему получилось плохое качество решения задачи?**

Скорее всего это связано с тем, что модель Word2Vec довольно старое решение, не учитывающее многие аспекты: контекст, плохая обработка редко встречающихся слов.

**Предложите свой подход к решению задачи.**

Улучшить результат можно использованием более современных моделей - GloVe и FastText, которые устраняют вышеописанные недостатки.